In [19]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tqdm import tqdm
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras import initializers

In [20]:
random_dim = 100

In [21]:
train_image = []

In [22]:
train_dir = '/Users/joannakang/Desktop/카카오_GAN/kakao_12_images'

In [23]:
img_list = os.listdir(train_dir)

In [24]:
print(img_list)

['8.jpg', '9.jpg', '12.jpg', '11.jpg', '10.jpg', '4.jpg', '5.jpg', '7.jpg', '6.jpg', '2.jpg', '3.jpg', '1.jpg']


In [25]:
for i in img_list:
    img = os.path.join(train_dir, i)
    img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (28, 28), interpolation = cv2.INTER_CUBIC)
    img = img.reshape((28, 28, 1))
    img = np.array(img)
    train_image.append(img)

In [26]:
len(train_image)

12

In [27]:
# cv2.IMREAD_COLOR
# cv2.IMREAD_GRAYSCALE

In [28]:
a = train_image[1].reshape(28, 28)

In [29]:
plt.imshow(a)

In [30]:
train_image = np.array(train_image)

In [31]:
train_image.shape

(12, 28, 28, 1)

In [32]:
def load_data():
      # 데이터를 -1 ~ 1 사이 값으로 normalize 합니다.
    x_train = (train_image.astype(np.float32) - 127.5)/127.5

    # x_train 의 shape 를 (60000, 28, 28) 에서 (60000, 784) 로 바꿉니다.
    # 따라서 우리는 한 row 당 784 columns 을 가지게 됩니다.
    x_train = x_train.reshape(train_image.shape[0], 784)
    return x_train

In [33]:
# Adam Optimizer를 사용합니다.
def get_optimizer():
    return Adam(lr=0.0002, beta_1=0.5)

# Generator 만들기
def get_generator(optimizer):
    generator = Sequential()
    generator.add(Dense(256, input_dim=random_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))

    generator.add(Dense(784, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return generator

# Discriminator 만들기
def get_discriminator(optimizer):
    discriminator = Sequential()
    discriminator.add(Dense(1024, input_dim=784, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(512))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(256))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)
    return discriminator

In [34]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # 우리는 Generator와 Discriminator를 동시에 학습시키고 싶을 때 trainable을 False로 설정합니다.
    discriminator.trainable = False

    # GAN 입력 (노이즈)은 위에서 100 차원으로 설정했습니다.
    gan_input = Input(shape=(random_dim,))

    # Generator의 결과는 이미지 입니다.
    x = generator(gan_input)

    # Discriminator의 결과는 이미지가 진짜인지 가짜인지에 대한 확률입니다.
    gan_output = discriminator(x)

    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [35]:
def plot_generated_images(epoch, generator, examples= train_image.shape[0], dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, random_dim])
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [36]:
def train(epochs=1, batch_size=6):
    # train 데이터와 test 데이터를 가져옵니다.
    x_train = load_data()

    # train 데이터를 128 사이즈의 batch 로 나눕니다.
    batch_count = x_train.shape[0] // batch_size

    # 우리의 GAN 네트워크를 만듭니다.
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        print('-'*15, 'Epoch %d' % e, '-'*15)
        for _ in tqdm(range(batch_count)):
            # 입력으로 사용할 random 노이즈와 이미지를 가져옵니다.
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])

            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # 이미지를 생성
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            y_dis = np.zeros(2*batch_size)
            y_dis[:batch_size] = 0.9

            # Discriminator를 학습
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Generator를 학습
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 20 == 0:
            plot_generated_images(e, generator)

if __name__ == '__main__':
    train(100, 128)

0it [00:00, ?it/s]

--------------- Epoch 1 ---------------



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

--------------- Epoch 2 ---------------
--------------- Epoch 3 ---------------
--------------- Epoch 4 ---------------
--------------- Epoch 5 ---------------
--------------- Epoch 6 ---------------
--------------- Epoch 7 ---------------
--------------- Epoch 8 ---------------
--------------- Epoch 9 ---------------
--------------- Epoch 10 ---------------
--------------- Epoch 11 ---------------
--------------- Epoch 12 ---------------
--------------- Epoch 13 ---------------
--------------- Epoch 14 ---------------
--------------- Epoch 15 ---------------
--------------- Epoch 16 ---------------
--------------- Epoch 17 ---------------
--------------- Epoch 18 ---------------
--------------- Epoch 19 ---------------
--------------- Epoch 20 ---------------



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

...

--------------- Epoch 41 ---------------
--------------- Epoch 42 ---------------
--------------- Epoch 43 ---------------
--------------- Epoch 44 ---------------
--------------- Epoch 45 ---------------
--------------- Epoch 46 ---------------
--------------- Epoch 47 ---------------
--------------- Epoch 48 ---------------
--------------- Epoch 49 ---------------
--------------- Epoch 50 ---------------
--------------- Epoch 51 ---------------
--------------- Epoch 52 ---------------
--------------- Epoch 53 ---------------
--------------- Epoch 54 ---------------
--------------- Epoch 55 ---------------
--------------- Epoch 56 ---------------
--------------- Epoch 57 ---------------
--------------- Epoch 58 ---------------
--------------- Epoch 59 ---------------
--------------- Epoch 60 ---------------



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

--------------- Epoch 61 ---------------
--------------- Epoch 62 ---------------
--------------- Epoch 63 ---------------
--------------- Epoch 64 ---------------
--------------- Epoch 65 ---------------
--------------- Epoch 66 ---------------
--------------- Epoch 67 ---------------
--------------- Epoch 68 ---------------
--------------- Epoch 69 ---------------
--------------- Epoch 70 ---------------
--------------- Epoch 71 ---------------
--------------- Epoch 72 ---------------
--------------- Epoch 73 ---------------
--------------- Epoch 74 ---------------
--------------- Epoch 75 ---------------
--------------- Epoch 76 ---------------
--------------- Epoch 77 ---------------
--------------- Epoch 78 ---------------
--------------- Epoch 79 ---------------
--------------- Epoch 80 ---------------



0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

--------------- Epoch 81 ---------------
--------------- Epoch 82 ---------------
--------------- Epoch 83 ---------------
--------------- Epoch 84 ---------------
--------------- Epoch 85 ---------------
--------------- Epoch 86 ---------------
--------------- Epoch 87 ---------------
--------------- Epoch 88 ---------------
--------------- Epoch 89 ---------------
--------------- Epoch 90 ---------------
--------------- Epoch 91 ---------------
--------------- Epoch 92 ---------------
--------------- Epoch 93 ---------------
--------------- Epoch 94 ---------------
--------------- Epoch 95 ---------------
--------------- Epoch 96 ---------------
--------------- Epoch 97 ---------------
--------------- Epoch 98 ---------------
--------------- Epoch 99 ---------------
--------------- Epoch 100 ---------------
